# Get all nouns

In [21]:
pip install pymorphy2

In [3]:
import csv
import re

import pandas as pd
import pymorphy2
morph = pymorphy2.MorphAnalyzer()

## 1. Make a dataframe out of csv file
And right away add a column for POS tags 

In [4]:
voc_cols = ['rus', 'agul']
df_voc = pd.read_csv('Agul_dict.csv', names=voc_cols)
df_voc['pos'] = ''

In [5]:
print(df_voc.head(2))

     rus          agul pos
0   бязь  агъ (ди, ар)    
1  бедро   агъ (у, ар)    


## 2. Get lists with nouns

In [13]:
good_nouns = []  # nouns for which all forms are goven
bad_nouns = []  # nouns for which only Sg Nom form is given

for index, row in df_voc.iterrows():
    p = morph.parse(row['rus'])[0]
    pos_ = p.tag.POS
    if pos_ == 'NOUN':  # since we need only nouns
        # first change capital "i" and "1" to Palochka ("ӏ"):
        row_agul = re.sub('[I1]', 'ӏ', row['agul'])
        if '(' in row['agul']:
            # for "good" nouns this cell looks like this: агъ (ди, ар)
            good_nouns.append([row['rus'], row_agul])
        else:
            # for "bad" nouns  this cell looks like this: азархана, — no parentheses
            bad_nouns.append([row['rus'], row['agul']])
    
    else:
        pass

## 3. Write nouns into csv files

In [18]:
def list_to_csv(list_: list, name_: str):
    df = pd.DataFrame(list_)
    df.columns = ['rus', 'agul']
    df = df[['agul', 'rus']]
    csv_name = name_ + '.csv'
    df.to_csv(csv_name, index=False)
    print(df.head(5))

In [19]:
list_to_csv(good_nouns, 'good_nouns')

                   agul     rus
0          агъ (ди, ар)    бязь
1           агъ (у, ар)   бедро
2        айван (ди, ар)  балкон
3  Аллагь (Аллагьди, -)     Бог
4       алухӏуб (а, ар)   брань


In [20]:
list_to_csv(bad_nouns, 'bad_nouns')

        agul       rus
0   азархана  больница
1  аттрухъуб      бред
2   ахттилат    беседа
3    багувел  близость
4       бала      беда
